### Preparing module and dataset

In [ ]:
!pip install --upgrade mxnet
!pip install autogluon
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown --id 1S692HG0MN9otHAHppD7yavr4VYN4D-Dc

In [ ]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [ ]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.0` and `torch==1.10.0+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/house-grade/
!mkdir /content/properties-tagging/

!unzip /content/drive/MyDrive/super-ai-engineer-2021-house-grade-classification.zip -d /content/house-grade/
!unzip /content/drive/MyDrive/super-ai-engineer-2021-properties-tagging.zip -d /content/properties-tagging/
!unzip /content/final_test.zip -d /content/house-grade/test

### Preparing dataset benchmark

In [ ]:
import pandas as pd
import os

In [ ]:
test_df=pd.read_csv('/content/properties-tagging/sample_submission.csv')
test_df

,Id,Labels
0,d5461e01,2 3
1,0e6bd6ea,0
2,03df66ca,3
3,bdf217f5,NaN
4,632726ba,NaN
...,...,...
1545,98768d3f,NaN
1546,34f08e48,NaN
1547,4337ff85,NaN
1548,825b04fe,NaN


In [ ]:
images=[]
image_dirs=[]
for image in os.listdir('/content/house-grade/test'):
  image_dir='/content/house-grade/test/'+image
  images.append(image[:-4])
  image_dirs.append(image_dir)

In [ ]:
test_df=pd.DataFrame({'image':image_dirs,'Id':images})
test_df

,image,Id
0,/content/house-grade/test/f0ea0a13.jpg,f0ea0a13
1,/content/house-grade/test/e035ad5b.jpg,e035ad5b
2,/content/house-grade/test/17daad98.jpg,17daad98
3,/content/house-grade/test/34b5c210.jpg,34b5c210
4,/content/house-grade/test/8f81b246.jpg,8f81b246
...,...,...
1545,/content/house-grade/test/5f033aaf.jpg,5f033aaf
1546,/content/house-grade/test/0188f57d.jpg,0188f57d
1547,/content/house-grade/test/6ee716ef.jpg,6ee716ef
1548,/content/house-grade/test/5e2d50a4.jpg,5e2d50a4


### Load model and predict each image

In [ ]:
model1=ImagePredictor.load('/content/drive/MyDrive/autogluon_model/predictor1.ag')
model2=ImagePredictor.load('/content/drive/MyDrive/autogluon_model/predictor2.ag')
model3=ImagePredictor.load('/content/drive/MyDrive/autogluon_model/predictor3.ag')
model4=ImagePredictor.load('/content/drive/MyDrive/autogluon_model/predictor4.ag')
model5=ImagePredictor.load('/content/drive/MyDrive/autogluon_model/predictor5.ag')

In [ ]:
predict_test1=model1.predict(pd.DataFrame(test_df['image']))
predict_test2=model2.predict(pd.DataFrame(test_df['image']))
predict_test3=model3.predict(pd.DataFrame(test_df['image']))
predict_test4=model4.predict(pd.DataFrame(test_df['image']))
predict_test5=model5.predict(pd.DataFrame(test_df['image']))

### Concatenate all labels into single column and submit prediction result

In [ ]:
predicts=[]
for index in range(len(test_df)):
  predict_list=[]
  if predict_test1[index]==1:
    predict_list.append('1')
  if predict_test2[index]==1:
    predict_list.append('2')
  if predict_test3[index]==1:
    predict_list.append('3')
  if predict_test4[index]==1:
    predict_list.append('4')
  if predict_test5[index]==1:
    predict_list.append('5')

  if predict_list==[]:
    predict_list=['0']

  predict_concat=' '.join(predict_list)
  predicts.append(predict_concat)

In [ ]:
submit_df=pd.read_csv('/content/properties-tagging/sample_submission.csv')
test_df['Labels']=predicts
submit_df=submit_df.merge(test_df,how='left',on='Id')
submit_df=submit_df.drop(['Labels_x','image'],axis=1).set_index('Id').rename(columns={'Labels_y':'Labels'})
submit_df

,Labels
Id,
d5461e01,0
0e6bd6ea,0
03df66ca,0
bdf217f5,2 4
632726ba,5
...,...
98768d3f,2 3 5
34f08e48,2 3
4337ff85,3


In [ ]:
submit_df.to_csv('Submission.csv')